# Build a Text2SQL AI Workflow with LangChain



In this project, we will design a Text2SQL workflow using LangChain, enabling users to convert natural language queries into SQL commands seamlessly. The workflow will consist of the following components:

Query Write Chain: A large language model (LLM) generates SQL queries based on the user's input question, adhering to constraints such as selecting relevant columns and limiting the number of results.

Query Execute Chain: The SQL query is executed against a pre-defined database schema using a database engine, retrieving the necessary results.

Answer Chain: The LLM formulates a user-friendly response, presenting the query results while escaping special characters for correct rendering in markdown.

By leveraging LangChain’s capabilities, we can simplify the process of interacting with databases, allowing users to obtain meaningful insights without requiring SQL expertise.

While this has agentic components like tool-calls we are defining a fixed deterministic flow where we have more control, however less flexibility.

![](https://i.imgur.com/7WrLz9I.png)



## Install OpenAI, and LangChain dependencies

In [ ]:
!pip install langchain==0.3.14
!pip install langchain-openai==0.3.0
!pip install langchain-community==0.3.14

In [ ]:
!apt-get install sqlite3 -y

## Enter Open AI API Key

In [ ]:
from getpass import getpass

OPENAI_KEY = getpass('Enter Open AI API Key: ')

## Setup Environment Variables

In [ ]:
import os

os.environ['OPENAI_API_KEY'] = OPENAI_KEY

## Get SQL DB Script

In [ ]:
# in case of issues download from https://drive.google.com/file/d/16mZm3C7xKpPqp_86e64uzduLpM5mPUdq/view?usp=sharing and upload
!gdown 16mZm3C7xKpPqp_86e64uzduLpM5mPUdq

## Create Comic Store Database

In [ ]:
!sqlite3 --version

In [ ]:
!sqlite3 ComicStore.db ".read ./comicdb_create_script.sql"

In [ ]:
!sqlite3 ComicStore.db "SELECT name FROM sqlite_master WHERE type='table';"

In [ ]:
%%bash
sqlite3 ComicStore.db <<EOF
.headers on
.mode column
SELECT * FROM Comic LIMIT 10;
EOF

In [ ]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///ComicStore.db")
db

In [ ]:
print(db.dialect)
print(db.get_usable_table_names())

## Overview of the Comic Store Database

In this project, we will utilize a **Comic Store Database** to demonstrate the capabilities of a Text2SQL workflow. The database schema includes the following entities and relationships:

1. **Branch**: Stores details about comic store branches, including their location and contact information.

2. **Publisher**: Contains information about publishers, such as their name, country, and the year they were established.

3. **Comic**: Represents the comics, including their title, genre, price, release date, and associated publisher.

4. **Customer**: Tracks customer details, including their contact information and location.

5. **Employee**: Holds data about store employees, including their branch, title, and hire date.

6. **Inventory**: Manages the stock of comics available at different branches.

7. **Sale**: Records sales transactions, including the employee and customer involved, as well as the total amount and sale date.

8. **SaleTransactions**: Tracks individual items within a sale, including the quantity and price of each comic sold.

### Relationships:
- A **Publisher** publishes multiple **Comics**.
- A **Branch** stocks multiple **Comics** through the **Inventory** table.
- A **Customer** makes **Sales**, which are processed by **Employees**.
- Each **Sale** contains multiple items recorded in **SaleTransactions**.

This database schema is well-suited for queries related to inventory management, sales analysis, customer interactions, and employee performance in the context of a comic store business.


![](https://i.imgur.com/YzNCLpV.png)

In [ ]:
db.run("SELECT * FROM Comic LIMIT 10;", include_columns=True)

In [ ]:
db.run("SELECT * FROM Employee LIMIT 10;", include_columns=True)

In [ ]:
db.run("SELECT * FROM Sale LIMIT 10;", include_columns=True)

In [ ]:
db.run("SELECT * FROM SaleTransactions LIMIT 10;", include_columns=True)

In [ ]:
print(db.get_table_info(table_names=['Comic', 'Sale']))

## Build Text2SQL Components for AI Workflow

In [34]:
# This prompt is customized from here
# https://github.com/langchain-ai/langchain/blob/master/libs/langchain/langchain/chains/sql_database/prompt.py
# you might need to customize it based on the LLM you are using
# the output format might vary so you may need to mention explicit instructions in the prompt

from langchain_core.prompts.prompt import PromptTemplate

PROMPT_SUFFIX = """Only use the following tables:
{table_info}

Question: {input}"""

_sqlite_prompt = """You are a SQLite expert.
Given an input question, first create a syntactically correct SQLite query to run,
then look at the results of the query and return the answer to the input question.

Unless the user specifies in the question a specific number of examples to obtain,
query for at most {top_k} results using the LIMIT clause as per SQLite.

You can order the results to return the most informative data in the database.
Never query for all columns from a table.

You must query only the columns that are needed to answer the question.
Wrap each column name in double quotes (") to denote them as delimited identifiers.

Pay attention to use only the column names you can see in the tables below.
Be careful to not query for columns that do not exist.
Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".
Pay attention to use table JOINS as necessary if you are adding relevant fields from different tables.

Generate the output in the exact following format:

SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

The SQLQuery field above should have the correct SQLite query as plain text without any formatting or code blocks.
Do not include sql or similar markers.
Do not try to explain the query, just provide the query as-is, like this: SELECT ...
"""

SQLITE_PROMPT = PromptTemplate(
    input_variables=["input", "table_info", "top_k"],
    template=_sqlite_prompt + PROMPT_SUFFIX,
)

In [35]:
SQLITE_PROMPT

PromptTemplate(input_variables=['input', 'table_info', 'top_k'], input_types={}, partial_variables={}, template='You are a SQLite expert.\nGiven an input question, first create a syntactically correct SQLite query to run,\nthen look at the results of the query and return the answer to the input question.\n\nUnless the user specifies in the question a specific number of examples to obtain,\nquery for at most {top_k} results using the LIMIT clause as per SQLite.\n\nYou can order the results to return the most informative data in the database.\nNever query for all columns from a table.\n\nYou must query only the columns that are needed to answer the question.\nWrap each column name in double quotes (") to denote them as delimited identifiers.\n\nPay attention to use only the column names you can see in the tables below.\nBe careful to not query for columns that do not exist.\nAlso, pay attention to which column is in which table.\nPay attention to use date(\'now\') function to get the cur

In [36]:
# Implementation details in langchain source code:
# https://api.python.langchain.com/en/latest/_modules/langchain/chains/sql_database/query.html#create_sql_query_chain
from langchain_openai import ChatOpenAI
from langchain.chains import create_sql_query_chain

chatgpt = ChatOpenAI(model="gpt-4o", temperature=0)
text2sql_chain = create_sql_query_chain(llm=chatgpt,
                                        db=db,
                                        prompt=SQLITE_PROMPT,
                                        k=5)
text2sql_chain

RunnableAssign(mapper={
  input: RunnableLambda(...),
  table_info: RunnableLambda(...)
})
| RunnableLambda(lambda x: {k: v for k, v in x.items() if k not in ('question', 'table_names_to_use')})
| PromptTemplate(input_variables=['input', 'table_info'], input_types={}, partial_variables={'top_k': '5'}, template='You are a SQLite expert.\nGiven an input question, first create a syntactically correct SQLite query to run,\nthen look at the results of the query and return the answer to the input question.\n\nUnless the user specifies in the question a specific number of examples to obtain,\nquery for at most {top_k} results using the LIMIT clause as per SQLite.\n\nYou can order the results to return the most informative data in the database.\nNever query for all columns from a table.\n\nYou must query only the columns that are needed to answer the question.\nWrap each column name in double quotes (") to denote them as delimited identifiers.\n\nPay attention to use only the column names you 

In [37]:
text2sql_chain.get_prompts()[0].pretty_print()

You are a SQLite expert.
Given an input question, first create a syntactically correct SQLite query to run,
then look at the results of the query and return the answer to the input question.

Unless the user specifies in the question a specific number of examples to obtain,
query for at most 5 results using the LIMIT clause as per SQLite.

You can order the results to return the most informative data in the database.
Never query for all columns from a table.

You must query only the columns that are needed to answer the question.
Wrap each column name in double quotes (") to denote them as delimited identifiers.

Pay attention to use only the column names you can see in the tables below.
Be careful to not query for columns that do not exist.
Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".
Pay attention to use table JOINS as necessary if you are adding relevant fields from differe

In [38]:
response = text2sql_chain.invoke({"question": "Top 5 most popular comics"})
print(response)

SELECT "Comic"."Title", SUM("SaleTransactions"."Quantity") AS "TotalSold"
FROM "SaleTransactions"
JOIN "Comic" ON "SaleTransactions"."ComicId" = "Comic"."ComicId"
GROUP BY "Comic"."Title"
ORDER BY "TotalSold" DESC
LIMIT 5


In [39]:
db.run(response)

"[('Wolverine: Old Man Logan', 3), ('V for Vendetta', 2), ('Usagi Yojimbo Volume 1', 2), ('Transformers: All Hail Megatron', 2), ('The Killing Joke', 2)]"

## Create SQL Query Write & Execute Workflow Chains

In [40]:
from langchain_community.tools import QuerySQLDatabaseTool

execute_query_tool = QuerySQLDatabaseTool(db=db)
execute_query_tool

QuerySQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x796a77cfd250>)

In [41]:
chatgpt = ChatOpenAI(model="gpt-4o", temperature=0)
query_write_chain = create_sql_query_chain(llm=chatgpt,
                                           db=db,
                                           prompt=SQLITE_PROMPT,
                                           k=10)
query_execute_chain = (query_write_chain
                            |
                       execute_query_tool)

query_execute_chain.invoke({"question": "Top 5 most popular comics"})

"[('Wolverine: Old Man Logan', 3), ('V for Vendetta', 2), ('Usagi Yojimbo Volume 1', 2), ('Transformers: All Hail Megatron', 2), ('The Killing Joke', 2)]"

In [42]:
query_execute_chain.invoke({"question": "Top 5 customers with most comics purchased"})

"[('Tony', 'Stark', 8), ('Sarah', 'Connor', 7), ('Natasha', 'Romanoff', 6), ('Clark', 'Kent', 6), ('Diana', 'Prince', 6)]"

In [43]:
query_execute_chain.invoke({"question": "Top 5 customers with most money spent"})

"[('Tony', 'Stark', 164.94), ('Bruce', 'Wayne', 139.94), ('Sarah', 'Connor', 124.96), ('Clark', 'Kent', 114.96), ('Natasha', 'Romanoff', 111.96000000000001)]"

In [44]:
query_execute_chain.invoke({"question": "Top 3 salesman with highest revenue"})

"[('John', 'Doe', 255.89), ('Alice', 'Brown', 234.94), ('Jane', 'Smith', 234.88)]"

## Create Text2SQL AI Workflow Chain

In [45]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result,
       create a helpful answer the user question.

       When generating the final answer in markdown from the results,
       if there are special characters in the text, such as the dollar symbol,
       ensure they are escaped properly for correct rendering e.g $25.5 should become \$25.5

       Question: {question}
       SQL Query: {query}
       SQL Result: {result}
       Answer:
    """
)

text2sql_chain = (
    RunnablePassthrough.assign(query=query_write_chain)
        |
    RunnablePassthrough.assign(result=query_execute_chain)
        |
    answer_prompt
        |
    chatgpt
        |
    StrOutputParser()
)

## Test the Text2SQL AI Workflow

In [46]:
from IPython.display import display, Markdown
response = text2sql_chain.invoke({"question": "Total number of customers"})
display(Markdown(response))

The total number of customers is **20**.

In [47]:
response = text2sql_chain.invoke({"question": "What are the Top 10 most popular comics"})
display(Markdown(response))

Here are the Top 10 most popular comics based on sales:

1. **Wolverine: Old Man Logan** - 3 copies sold
2. **V for Vendetta** - 2 copies sold
3. **Usagi Yojimbo Volume 1** - 2 copies sold
4. **Transformers: All Hail Megatron** - 2 copies sold
5. **The Killing Joke** - 2 copies sold
6. **The Boys Volume 1** - 2 copies sold
7. **Superman: Red Son** - 2 copies sold
8. **Punisher: Welcome Back, Frank** - 2 copies sold
9. **Preacher Volume 1** - 2 copies sold
10. **Ms. Marvel Volume 1** - 2 copies sold

These comics have proven to be the most popular based on the number of copies sold.

In [48]:
response = text2sql_chain.invoke({"question": "Top 5 customers with most comics purchased"})
display(Markdown(response))

Here are the top 5 customers who have purchased the most comics:

1. **Tony Stark** - 8 comics
2. **Sarah Connor** - 7 comics
3. **Natasha Romanoff** - 6 comics
4. **Clark Kent** - 6 comics
5. **Diana Prince** - 6 comics

These customers have shown a great interest in comics, with Tony Stark leading the list with 8 purchases.

In [49]:
response = text2sql_chain.invoke({"question": "Which are the top 5 customers with most money spent"})
display(Markdown(response))

Here are the top 5 customers who have spent the most money:

1. **Tony Stark**: \$164.94
2. **Bruce Wayne**: \$139.94
3. **Sarah Connor**: \$124.96
4. **Clark Kent**: \$114.96
5. **Natasha Romanoff**: \$111.96

These customers have made significant purchases, contributing to their high total spending.

In [50]:
response = text2sql_chain.invoke({"question": "Which are the top 3 salesman with highest revenue"})
display(Markdown(response))

The top 3 salespeople with the highest revenue are:

1. **John Doe** with a total revenue of \$255.89
2. **Alice Brown** with a total revenue of \$234.94
3. **Jane Smith** with a total revenue of \$234.88

These individuals have achieved the highest sales figures in the company.